In [1]:
import pandas as pd
from __future__ import division

## 计算器
### 一. 计算器的实现有几个问题  
#### 1. 读取操作数:  
因为表达式中的操作数可能是多位实数.这就要求读到一个数字时,先不能参与运算,要进入一个`操作数缓存栈`中,直到读取的表达式字符不是数字时,再把这个操作数缓存栈中的数字一次弹出\*${10}^n$,结果加入真正的`操作数栈`  
#### 2. 何时执行操作符运算:  
读到操作符时也不能直接运算,因为后面可能有操作级别更高的操作符. 此时只能先把操作符进入`操作符栈`,直到遇到表达式中的字符优先级,比`操作符栈`栈顶元素的优先级低时,才能对操作符栈顶的运算符运算 (只要出现的操作符比前一个操作符优先级低,就可以对前一个运算符进行计算)
#### 3. 对于括号的处理
1. '('前面的式子可以单独计算, 因此'('的优先级对比其他运算符低. 
 而'('里先遇到的运算符也不能直接运算, 因为括号内也是作为独立的式子运算, 因此其他运算符对比'('的优先级低
2. ')'的操作 
 1. ')'的优先级比所有有意义的操作符都低, 可执行栈顶操作符运算
 2. 若果当前运算符是')', 遇到'('时, 弹栈. 为了识别这种情况, 让')'与'('比较的结果为2

In [2]:
# 定有运算符优先级表. 1表示优先级高
#   横轴 : 当前运算符
#   纵轴 : 栈顶运算符
opt_prior = pd.DataFrame([[0,0,0,0,1,1],
                          [0,0,0,0,1,1],
                          [1,1,0,0,1,1],
                          [1,1,0,0,1,1],
                          [1,1,1,1,1,1],
                          [0,0,0,0,2,1],
                          [0,0,0,0,0,2]
                          ],
                         columns=['+','-','*','/','(','#'],
                         index=  ['+','-','*','/','(',')','#'])
opt_prior.index.name = 'current'
opt_prior.columns.name = 'stack_top'
opt_prior

stack_top,+,-,*,/,(,#
current,,,,,,
+,0,0,0,0,1,1
-,0,0,0,0,1,1
*,1,1,0,0,1,1
/,1,1,0,0,1,1
(,1,1,1,1,1,1
),0,0,0,0,2,1
#,0,0,0,0,0,2


In [3]:
def read_num(stack):
    '''从操作数缓存栈读取操作数'''
    flag = True
    if len(stack)==0:
        flag = False
    tmp = 0
    for index,n in enumerate(range(len(stack)-1,-1,-1)): #从(n-1)到0的逆序
        tmp = tmp + int(stack[index])*(10**n)
    return flag,tmp

In [4]:
def pop(stack):
    '''弹栈'''
    res = stack[-1]
    stack = stack[:-1]
    return res,stack
def caculate(caculator,stack):
    '''弹出栈顶2个字符'''
    print('stack:',stack,'caculator:',caculator)
    elem1 = stack[-2]
    elem2 = stack[-1]
    stack = stack[:-2]
    if caculator=='+':
        res = elem2 + elem1
    if caculator=='-':
        res = elem2-elem1
    if caculator=='*':
        res = elem2 * elem1
    if caculator=='/':
        res = elem2/elem1
    stack.insert(0,res)
    return stack

In [5]:
def evaluate(expression):
    # 表达式先补#, 便于匹配
    expression = list(expression)
    expression.append('#')
    number_cache = []
    numbers = []
    operaters = ['#']
    idx = 0 #指向表达式的当前字符
    while len(operaters)!=0:
        c = expression[idx] # 查看当前表达式的字符,idx在后面分情况是否后移
        if c.isdigit(): # 数字
            number_cache.append(c)
            idx = idx+1
        else: # 操作符
            # 1.遇到操作符时, 先从操作数缓存栈中提取操作数
            flag,number = read_num(number_cache)
            if flag:
                numbers.append(number)
            number_cache = []
            # 2.根据操作符选择进行的操作
            stack_top = operaters[-1]
            if opt_prior.loc[c,stack_top]==0: # 当前操作符优先级比栈顶优先级低, 则可计算栈顶操作
                calculator,operaters = pop(operaters)
                numbers = caculate(calculator,numbers) # 计算
            if opt_prior.loc[c,stack_top]==1: # 当前操作符优先级更高
                operaters.append(c)
                idx = idx + 1
            if opt_prior.loc[c,stack_top]==2:
                # 有2种情况导致比较结果为2. 左右括号比较或者2个\0比较
                _,operaters = pop(operaters)
                idx = idx+1
    # 返回运算结果
    return numbers[0]

In [6]:
s = '3*3+2+(1+2*3)'
evaluate(s)

stack: [3, 3] caculator: *
stack: [9, 2] caculator: +
stack: [11, 1, 2, 3] caculator: *
stack: [6, 11, 1] caculator: +
stack: [12, 6] caculator: +


18